In [31]:
import pandas as pd
data=pd.read_csv('processed_data/data.csv')
import nltk
from nltk.corpus import stopwords
stopwords.words('english')

In [137]:
data=data[data.sender.isin(data.groupby('sender').size().pipe(lambda x: x[x>100]).index)]

In [152]:
splitted=data.body.str.lower().str.replace('[^a-z ]', '').str.split().dropna()

In [153]:
data

,sender,body
64,andy.zipper@enron.com,I would like Amita to have the ability to add ...
65,andy.zipper@enron.com,I think the broker is never really the counter...
66,andy.zipper@enron.com,"Let me try and be clear on this, as I m not su..."
67,andy.zipper@enron.com,FYI
68,andy.zipper@enron.com,Mark can we put togethr a term sheet for the b...
69,andy.zipper@enron.com,FYI
70,andy.zipper@enron.com,Try this one....
71,andy.zipper@enron.com,"This is what I got back from espeed, which is ..."
72,andy.zipper@enron.com,The going gets pretty slow with all the differ...
73,andy.zipper@enron.com,Please take a look at these prior to us gettin...


In [154]:
def b(x):
    try:
        return len(x)
    except:
        print(x)
        return None
splitted.apply(b)

64        43
65        62
66       226
67         1
68        74
69         1
70         3
71        23
72        10
73        11
74        31
75        11
76        19
77        12
78        13
79        74
80         1
81        10
82        11
83        31
84        19
85        12
86        13
87        11
88         4
89        21
90         8
91        56
92        57
93        18
        ... 
86894     19
86895      5
86896     11
86897     12
86898      5
86899     11
86900     10
86901     15
86902      1
86903      5
86904     16
86905     19
86906     75
86907     11
86908     32
86909      7
86910      5
86911     47
86912     12
86913      9
86914     14
86915    286
86916      1
86917     17
86918     11
86919     75
86920      7
86921      3
86922     46
86923     10
Name: body, Length: 86350, dtype: int64

In [155]:
from itertools import chain
from collections import Counter
def get_word_count(x):
    try:
        counted=Counter(chain(*x))
        counter_words=pd.Series(counted)/len(x)
    except:
        print(x)
        raise
    return counter_words

In [156]:
words=get_word_count(splitted).to_frame().rename(columns={0:'baseline'}).query(f'baseline>5/{len(splitted)}')

In [157]:
words_by_sender=data.assign(splitted=splitted).dropna().groupby('sender').splitted.apply(get_word_count).unstack()

In [158]:
rel_count=pd.merge(words_by_sender.T.reset_index(), words.reset_index(), on="index").set_index(
    "index"
).apply(lambda x: x / x.baseline, axis=1)

In [159]:
# Words per email-person, eg how many times does X say W per email, on average
# vs Words per email, how many times an emails says W, on average
%matplotlib inline
rel_count

,andy.zipper@enron.com,barry.tycholiz@enron.com,benjamin.rogers@enron.com,bill.rapp@enron.com,cara.semperger@enron.com,carol.clair@enron.com,charles.weldon@enron.com,chris.dorland@enron.com,chris.germany@enron.com,dan.hyvl@enron.com,...,steven.j.kean@enron.com,susan.bailey@enron.com,susan.scott@enron.com,tana.jones@enron.com,teb.lokey@enron.com,theresa.staab@enron.com,thomas.a.martin@enron.com,tori.kuykendall@enron.com,tracy.geaccone@enron.com,baseline
index,,,,,,,,,,,,,,,,,,,,,
a,0.655751,0.981237,0.779909,0.896421,1.292138,1.198557,1.140530,0.792576,0.798849,1.514127,...,1.085551,0.815398,2.148844,0.962905,0.410672,0.335162,0.656941,1.011896,0.192167,1.0
aa,NaN,1.112872,0.608836,NaN,NaN,1.210210,3.584698,NaN,NaN,NaN,...,0.300876,NaN,2.254788,0.152166,NaN,NaN,NaN,NaN,NaN,1.0
aaa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,6.451854,NaN,NaN,NaN,NaN,NaN,1.0
aadepartment,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,47.839335,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
aaron,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.739444,NaN,...,3.776790,NaN,NaN,NaN,NaN,NaN,NaN,9.089474,NaN,1.0
ab,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.188291,NaN,NaN,...,1.203505,NaN,NaN,0.152166,NaN,NaN,NaN,NaN,NaN,1.0
aba,NaN,NaN,NaN,NaN,NaN,6.872264,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,6.912701,NaN,NaN,NaN,NaN,NaN,1.0
abandon,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,3.012700,NaN,NaN,NaN,NaN,NaN,NaN,1.0
abandoned,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


In [194]:
#Not removing stopwords, I or please might be useful
rel_count.isnull().mean(axis=1).pipe(lambda x: x[x<0.01])

index
a         0.0
about     0.0
all       0.0
am        0.0
an        0.0
and       0.0
are       0.0
as        0.0
at        0.0
back      0.0
be        0.0
been      0.0
but       0.0
by        0.0
can       0.0
day       0.0
did       0.0
do        0.0
for       0.0
get       0.0
going     0.0
good      0.0
have      0.0
here      0.0
i         0.0
if        0.0
in        0.0
is        0.0
it        0.0
just      0.0
         ... 
or        0.0
out       0.0
please    0.0
see       0.0
so        0.0
thanks    0.0
that      0.0
the       0.0
there     0.0
they      0.0
think     0.0
this      0.0
time      0.0
to        0.0
today     0.0
until     0.0
up        0.0
us        0.0
want      0.0
was       0.0
we        0.0
what      0.0
when      0.0
who       0.0
will      0.0
with      0.0
work      0.0
would     0.0
you       0.0
your      0.0
Length: 73, dtype: float64

In [213]:
ignore_words=rel_count.isnull().mean(axis=1).pipe(lambda x: x[x>0.8]).index
rel_count[~rel_count.index.isin(ignore_words)].loc['cute'].sort_values()

jeff.dasovich@enron.com         0.382024
james.d.steffes@enron.com       0.508474
carol.clair@enron.com           0.586657
eric.bass@enron.com             0.766969
debra.perlingiere@enron.com     0.859632
kimberly.watson@enron.com       0.918891
baseline                        1.000000
gerald.nemec@enron.com          1.002904
kay.mann@enron.com              1.123851
daren.j.farmer@enron.com        1.207625
matthew.lenhart@enron.com       1.238395
shelley.corman@enron.com        1.464602
john.arnold@enron.com           1.501139
sally.beck@enron.com            2.044755
kevin.hyatt@enron.com           2.064802
marie.heard@enron.com           2.259761
tana.jones@enron.com            2.360434
scott.neal@enron.com            2.404221
jane.tholt@enron.com            2.568412
mark.whitt@enron.com            2.941477
lindy.donoho@enron.com          3.441336
chris.germany@enron.com         3.808495
larry.f.campbell@enron.com      4.026955
stacy.dickson@enron.com         4.387703
stanley.horton@e

In [252]:
coordial = ["thanks", "thanks", "pls"]
#Issue: "not good"
nice = ["cute", "nice", "nicely", "nicest", "funny","like","good"]
shorts = ["no", "yes", "fyi","approved","not"]
social = [
    "drink",
    "drinks",
    "drinking",
    "horse",
    "horses",
    "fun",
    "funeral",
    "wedding",
    "weddings",
    "child",
    "children",
    "grandchild",
    "grandchildren",
    "football"
]
bad = ["hate","dislike"]
tentative=['perhaps','maybe','could']
work=['busy','work']

In [264]:
email_metadata=data.assign(
    email_len=lambda x: x.body.str.len(), is_short=lambda x: x.body.str.len() < 10
).drop(columns="body").groupby("sender").mean().reset_index().rename(
    columns={"sender": "index"}
).set_index(
    "index"
)

In [296]:
X=rel_count[rel_count.index.isin(coordial+nice+shorts+social+bad+tentative+work)].fillna(0).T
X_norm=X.join(email_metadata).drop('baseline').apply(lambda x: (x-x.mean())/x.std(),axis=0)
from sklearn.decomposition import PCA,FactorAnalysis
pca=FactorAnalysis(2)
pca.fit(X_norm)

FactorAnalysis(copy=True, iterated_power=3, max_iter=1000, n_components=2,
        noise_variance_init=None, random_state=0, svd_method='randomized',
        tol=0.01)

In [280]:
X_norm

,approved,busy,child,children,could,cute,drink,drinking,drinks,football,...,not,perhaps,pls,thanks,wedding,weddings,work,yes,email_len,is_short
andy.zipper@enron.com,-0.459075,-0.512331,-0.199271,0.621933,-0.609117,-0.388002,-0.439191,-0.447014,-0.589275,-0.449187,...,-0.484205,-0.168333,-0.066490,-0.259235,-0.394646,-0.173923,-0.279599,-0.207237,-0.875882,0.372656
barry.tycholiz@enron.com,-0.219100,0.001595,-0.199271,-0.303445,-0.616304,-0.388002,0.770548,-0.447014,0.062785,-0.449187,...,0.148106,2.324368,7.524547,0.238163,0.166370,-0.173923,0.559810,0.482264,-0.324682,-0.527250
benjamin.rogers@enron.com,-0.415313,0.452552,-0.199271,-0.303445,-0.299835,-0.388002,-0.273734,-0.447014,-0.232542,-0.449187,...,-0.660552,-0.541131,-0.180161,0.160076,1.446897,-0.173923,0.513800,0.092738,-0.626013,-0.840084
bill.rapp@enron.com,-0.459075,0.017973,-0.199271,-0.303445,-0.688994,-0.388002,-0.439191,-0.447014,-0.589275,-0.449187,...,0.296753,0.253394,-0.180161,1.878593,-0.394646,-0.173923,-1.145460,0.253003,-0.429015,0.249720
cara.semperger@enron.com,-0.107400,-0.981109,-0.199271,-0.303445,-0.591980,-0.388002,-0.439191,-0.447014,-0.589275,-0.449187,...,0.330494,-0.541131,-0.180161,-0.291105,-0.394646,-0.173923,0.861930,-0.740919,0.167930,-1.046805
carol.clair@enron.com,0.345564,-0.268671,-0.021121,-0.303445,1.286165,-0.211354,-0.439191,-0.447014,-0.412002,-0.449187,...,0.298400,0.379544,-0.180161,-0.569153,-0.394646,-0.173923,0.798874,-0.919980,0.923709,-1.021123
charles.weldon@enron.com,-0.459075,-0.981109,-0.199271,-0.303445,0.015534,-0.388002,-0.439191,-0.447014,-0.589275,0.158623,...,0.080118,0.297971,-0.180161,-0.312217,-0.394646,-0.173923,-0.670188,-1.095743,0.472405,0.322463
chris.dorland@enron.com,-0.459075,1.484493,-0.199271,-0.303445,-0.123939,-0.388002,1.078425,3.429144,0.228734,-0.449187,...,-1.222932,0.112463,-0.180161,-0.821748,1.012943,3.660973,0.173700,-1.141093,-0.885141,-0.928299
chris.germany@enron.com,-0.035539,-0.354801,-0.102894,0.267177,-0.485481,0.758774,-0.261267,0.189200,-0.589275,-0.282671,...,-0.253253,-0.464504,-0.180161,-0.461697,0.059172,0.725279,-0.086873,-0.439171,-0.241891,-0.525798
dan.hyvl@enron.com,0.289400,-0.509566,-0.199271,-0.070736,0.740469,-0.388002,-0.003828,0.442559,-0.589275,-0.449187,...,1.814970,-0.541131,-0.180161,-0.638677,0.816752,-0.173923,-0.101835,0.165912,1.618057,-0.570860


In [291]:
import matplotlib.pyplot as plt
pd.DataFrame(pca.transform(X_norm))

,0,1
0,-1.720241,-0.412218
1,-0.228142,-0.909912
2,-0.284819,0.789166
3,-1.925274,-0.804520
4,-0.757649,-0.981451
5,0.172321,-1.430617
6,-0.686593,-0.710373
7,1.375071,3.525260
8,-0.240897,0.538840
9,0.939917,-0.946041


In [298]:
#Loadings
pd.DataFrame(pca.components_)

,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
0,0.051842,0.487994,0.210605,0.269249,0.585481,0.276941,0.157070,0.164275,0.240922,0.275298,...,0.732968,0.270870,0.016324,0.033328,0.165919,0.148943,0.657464,0.156683,0.785129,-0.398739
1,0.278127,-0.551449,-0.129425,0.028964,0.229870,-0.326341,-0.510012,-0.296425,-0.580140,-0.221498,...,0.358713,0.279559,0.070497,0.123219,-0.211639,-0.186368,-0.024173,-0.200618,0.412021,0.106721


In [282]:
pd.DataFrame(pca.fit_transform(X)).sort_values(0)

,0,1
1,-4.266067,-1.396579
17,-4.187744,-1.193337
7,-3.682295,-10.190613
70,-3.442466,-1.515944
77,-3.160304,-0.307616
88,-3.143988,-0.007452
13,-3.050733,-0.187294
31,-3.018074,-0.930242
9,-2.998902,-0.029026
29,-2.960271,-0.036956
